In [1]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from hummingbot.strategy_v2.utils.distributions import Distributions
from controllers.market_making.pmm_simple import PMMSimpleConfig
from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from decimal import Decimal


class PMMSimpleConfigGenerator(BaseStrategyConfigGenerator):
    """
    Strategy configuration generator for PMM Simple optimization.
    """
    async def generate_config(self, trial) -> BacktestingConfig:
        # Controller configuration
        connector_name = "binance_perpetual"
        trading_pair = "1000PEPE-USDT"
        total_amount_quote = 1000
        take_profit = trial.suggest_float("take_profit", 0.02, 0.05, step=0.01)
        stop_loss = trial.suggest_float("stop_loss", 0.005, 0.05, step=0.005)
        levels = trial.suggest_int("levels", 3, 5)
        start_spread = trial.suggest_float("start_spread", 0.002, 0.005, step=0.001)
        step_spread = trial.suggest_float("step_spread", 0.001, 0.002, step=0.001)
        spreads = Distributions.arithmetic(levels, start_spread, step_spread)
        trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.005, 0.015, step=0.01)
        trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.05, 0.1, step=0.01)
        trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        time_limit = trial.suggest_int("time_limit", 60 * 60 * 2, 60 * 60 * 4, step=60 * 60)
        executor_refresh_time = trial.suggest_int("executor_refresh_time", 60 * 2, 60 * 10, step=60)
        cooldown_time = trial.suggest_int("cooldown_time", 60 * 5, 60 * 10, step=60)


        # Creating the instance of the configuration and the controller
        config = PMMSimpleConfig(
            connector_name=connector_name,
            trading_pair=trading_pair,
            sell_spreads=spreads,
            buy_spreads=spreads,
            total_amount_quote=Decimal(total_amount_quote),
            take_profit=Decimal(take_profit),
            stop_loss=Decimal(stop_loss),
            trailing_stop=TrailingStop(activation_price=Decimal(trailing_stop_activation_price), trailing_delta=Decimal(trailing_stop_trailing_delta)),
            time_limit=time_limit,
            cooldown_time=cooldown_time,
            executor_refresh_time=executor_refresh_time
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [5]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime

start_date = datetime.datetime(2024, 8, 1)
end_date = datetime.datetime(2024, 8, 3)
config_generator = PMMSimpleConfigGenerator(start_date=start_date, end_date=end_date)

optimizer = StrategyOptimizer(root_path=root_path)
await optimizer.optimize(
    study_name="pmm_simple_with_optimizer",
    config_generator=config_generator,
    n_trials=100,
)

[I 2024-09-05 15:24:47,765] Using an existing study with name 'pmm_simple_with_optimizer' instead of creating a new one.
2024-09-05 15:24:49,155 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13f399510>
2024-09-05 15:24:49,157 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13f45c340>, 33883.27009025)]']
connector: <aiohttp.connector.TCPConnector object at 0x13f399e40>
127.0.0.1 - - [05/Sep/2024 15:24:49] "GET /api/studies/2?after=1 HTTP/1.1" 200 58196
127.0.0.1 - - [05/Sep/2024 15:25:00] "GET /api/studies/2?after=1 HTTP/1.1" 200 134073
127.0.0.1 - - [05/Sep/2024 15:25:12] "GET /api/studies/2?after=1 HTTP/1.1" 200 214237
127.0.0.1 - - [05/Sep/2024 15:25:24] "GET /api/studies/2?after=1 HTTP/1.1" 200 290014
127.0.0.1 - - [05/Sep/2024 15:25:36] "GET /api/studies/2?after=1 HTTP/1.1" 200 365903
127.0.0.1 - - [05/Sep/2024 15:25:51] "GET /api/studies/2?after=1 HTTP/1.1" 200 456872


In [4]:
optimizer.launch_optuna_dashboard()